# Upload CrossRef to a local database

CrossRef have kindly made all of their 'works' database of publication data available for free via bit-torrent. Once we have downloaded the data, it's a good idea to upload it to a database so that we can 'index' the data. 

_Indexing_ gives the database a framework to 'look up' records instantly instead of having to scan over the whole database to look for things. They also allow us to create custom queries, like 'get all the records for journal articles published by a particular publisher in 2020'. That kind of thing.

# Acquire data
The first step here is to get the CrossRef data dump. You will need a bit-torrent client and then follow the instructions in the <a href = 'https://www.crossref.org/blog/free-public-data-file-of-112-million-crossref-records/'>CrossRef Blog</a>

In [1]:
# import all the packages we need
import pymongo
import json
import gzip
import glob
from datetime import datetime

### First, take a look at the data. What does it look like?

In [2]:
# list all the files in our data-dump
loc = r"D:\crossref\datadump\crossref\*"
filepaths = glob.glob(loc)

In [17]:
# example filepath
filepaths[0]

'D:\\crossref\\datadump\\crossref\\0.json.gz'

In [4]:
# example open a file and look inside
with gzip.open(filepaths[0], 'r') as f:
    data = json.load(f)

In [5]:
len(data['items'])

3000

We find that the data is one big JSON object with 'items' as a key and then a list of 3000 CrossRef 'works' records.  Works records generally pertain to published research papers, but occasionally other documents, too. 

In [15]:
# example works record
data['items'][0]

{'indexed': {'date-parts': [[2019, 11, 19]],
  'date-time': '2019-11-19T17:15:40Z',
  'timestamp': 1574183740979},
 'reference-count': 0,
 'publisher': 'American Medical Association (AMA)',
 'issue': '4',
 'content-domain': {'domain': [], 'crossmark-restriction': False},
 'short-container-title': ['Archives of Internal Medicine'],
 'published-print': {'date-parts': [[2006, 2, 27]]},
 'DOI': '10.1001/.389',
 'type': 'journal-article',
 'created': {'date-parts': [[2006, 2, 27]],
  'date-time': '2006-02-27T21:28:23Z',
  'timestamp': 1141075703000},
 'page': '389-390',
 'source': 'Crossref',
 'is-referenced-by-count': 0,
 'title': ['Decision Making at the Fringe of Evidence: Take What You Can Get'],
 'prefix': '10.1001',
 'volume': '166',
 'author': [{'given': 'N. F.', 'family': 'Col', 'affiliation': []}],
 'member': '10',
 'container-title': ['Archives of Internal Medicine'],
 'original-title': [],
 'deposited': {'date-parts': [[2007, 2, 13]],
  'date-time': '2007-02-13T20:56:13Z',
  'tim

If we assume that the structure of all of the JSON files in our data dump is the same, then we can build a function to load all of that data into our database.

In [6]:
# define a class to iterate through the data

class CRDump():
    loc = r"D:\crossref\datadump\crossref\*"
    filepaths = glob.glob(loc)
    def __iter__(self):
        """
        The class is an object a bit like a list - in that we can iterate over it.
        This function defines how that iteration happens. 
        """
        for filepath in filepaths:
            with gzip.open(filepath, 'r') as f:
                data = json.load(f)
                items = data['items']
                for item in items:
                    try:
                        doi = item['DOI']
                        yield doi,item
                    except:
                        print('Fail')
                        print(item)
                        pass
                
cr_iterator = CRDump()

# Dump into MongoDB

MongoDB is a popular NoSQL database which is useful for storing and indexing large quantities of JSON data. 

Other databases may also be worth trying:
- Other NoSQL databases include neo4j, or elasticsearch. These are recommended if you need to do complex queries on the data.
- PostgresSQL also allows indexing of JSON as well as being a sophisticated SQL db

## Setting up MongoDB
First, <a href='https://www.mongodb.com/community'>download and install MongoDB community edition.</a>

You will need to create a folder 'data\db' in the top-level of a hard drive. E.g. create `D:\data\db`, or `C:\data\db`

Finally, you might need to start a MongoDB server, by opening a cmd window and entering `mongod`

In [8]:
# create a client (this is your connection to the mongod service you started)
client = pymongo.MongoClient("mongodb://localhost:27017/")
# create or connect to a mongodb database via the client
db = client["crossref"]
# create or connect to a table (or 'collection') in that mongodb database
collection = db["works"]

In [9]:
# check which databases already exist - note that some dbs are created by default when you install mongodb
client.database_names()

C:\Users\AdamD\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  """Entry point for launching an IPython kernel.


['admin', 'config', 'crossref', 'local']

In [10]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'crossref'), 'works')

# Upload data to MongoDB
- pick a batch size to upload records in (I find 1000 seems optimal)
- then run this cell to upload the data to MongoDB. <span style="color:red">This took around 11+1/2 hrs on my home computer, so be prepared to wait!</span>

In [11]:
%%time
batch_size = 1000
batch = []
i=0
for doi, works_record in cr_iterator:
    i+=1
    works_record['_uploaded'] = datetime.strftime(datetime.now(), "%Y-%m-%d")
    batch.append(pymongo.UpdateOne({'DOI':doi}, {'$setOnInsert':works_record}, upsert = True))
    if i%batch_size==0:
        result = collection.bulk_write(batch)
        batch = []
        if i%100000==0:
            print(datetime.now(),i, ' records written to Mongo')
result = collection.bulk_write(batch)


2020-04-14 14:30:41.158039 100000  records written to Mongo
2020-04-14 14:31:09.000146 200000  records written to Mongo
2020-04-14 14:31:35.101790 300000  records written to Mongo
2020-04-14 14:32:00.490812 400000  records written to Mongo
2020-04-14 14:32:27.754762 500000  records written to Mongo
2020-04-14 14:32:51.848780 600000  records written to Mongo
2020-04-14 14:33:18.059584 700000  records written to Mongo
2020-04-14 14:33:41.828271 800000  records written to Mongo
2020-04-14 14:34:07.855025 900000  records written to Mongo
2020-04-14 14:34:33.314709 1000000  records written to Mongo
2020-04-14 14:34:58.051775 1100000  records written to Mongo
2020-04-14 14:35:22.821386 1200000  records written to Mongo
2020-04-14 14:35:47.859687 1300000  records written to Mongo
2020-04-14 14:36:12.015898 1400000  records written to Mongo
2020-04-14 14:36:38.383670 1500000  records written to Mongo
2020-04-14 14:37:04.656983 1600000  records written to Mongo
2020-04-14 14:37:29.772917 170000

# Add indexes
Now we need to add indexes. see: https://docs.mongodb.com/manual/indexes/

#### Index on
- DOI
- publication date (note that CrossRef stores several dates for each entry - you might need multiple indexes)
- Publisher
- ISSN / eISSN

In [16]:
%%time
db.works.create_index([( 'DOI', pymongo.ASCENDING )])

In [13]:
%%time
db.works.create_index([( 'issn-type.value', pymongo.ASCENDING )])

'issn-type.value_1'

# Check

In [18]:
# look up article by DOI
collection.find_one({'DOI':'10.1177/2158244017712774'})

{'_id': ObjectId('5e9600dd96e4f11cab42b892'),
 'DOI': '10.1177/2158244017712774',
 'ISSN': ['2158-2440', '2158-2440'],
 'URL': 'http://dx.doi.org/10.1177/2158244017712774',
 '_uploaded': '2020-04-14',
 'alternative-id': ['10.1177/2158244017712774'],
 'author': [{'given': 'Logan S.',
   'family': 'Casey',
   'affiliation': [{'name': 'Harvard T.H. Chan School of Public Health, Boston, MA, USA'}]},
  {'given': 'Jesse',
   'family': 'Chandler',
   'affiliation': [{'name': 'University of Michigan, Ann Arbor, USA'},
    {'name': 'Mathematica Policy Research, Ann Arbor, MI, USA'}]},
  {'given': 'Adam Seth',
   'family': 'Levine',
   'affiliation': [{'name': 'Cornell University, Ithaca, NY, USA'}]},
  {'given': 'Andrew',
   'family': 'Proctor',
   'affiliation': [{'name': 'Princeton University, NJ, USA'}]},
  {'given': 'Dara Z.',
   'family': 'Strolovitch',
   'affiliation': [{'name': 'Princeton University, NJ, USA'}]}],
 'container-title': ['SAGE Open'],
 'content-domain': {'domain': ['journa

In [21]:
# look up articles by issn and year.  Easy.
issn = '2158-2440'
year = 2019
# create a cursor - essentially an object which represents our 'lookup' of the data
cur = collection.find({'issn-type.value':issn,
                    'issued.date-parts.0.0':year})
# read the data from that 'lookup' into a list object
works = [x for x in cur]
# how many did we get?
len(works)

393

In [23]:
# inspect the data manually
works[:3]

[{'_id': ObjectId('5e9600dd96e4f11cab42b9b4'),
  'DOI': '10.1177/2158244018789229',
  'ISSN': ['2158-2440', '2158-2440'],
  'URL': 'http://dx.doi.org/10.1177/2158244018789229',
  '_uploaded': '2020-04-14',
  'alternative-id': ['10.1177/2158244018789229'],
  'author': [{'given': 'Andrea',
    'family': 'Ceron',
    'affiliation': [{'name': 'Università degli Studi di Milano, Italy'}]},
   {'given': 'Luigi',
    'family': 'Curini',
    'affiliation': [{'name': 'Università degli Studi di Milano, Italy'}]},
   {'ORCID': 'http://orcid.org/0000-0002-4884-0047',
    'authenticated-orcid': False,
    'given': 'Stefano M.',
    'family': 'Iacus',
    'affiliation': [{'name': 'Università degli Studi di Milano, Italy'}]}],
  'container-title': ['SAGE Open'],
  'content-domain': {'domain': ['journals.sagepub.com'],
   'crossmark-restriction': True},
  'created': {'date-parts': [[2019, 3, 27]],
   'date-time': '2019-03-27T17:26:18Z',
   'timestamp': 1553707578000},
  'deposited': {'date-parts': [[20